In [2]:
# Import modules

import spiceypy
import datetime
import numpy as np

# Load the SPICE kernels

spiceypy.furnsh("../kernels/lsk/naif0012.tls")
spiceypy.furnsh("../kernels/spk/de432s.bsp")
# Kernel for gravitational parameters
spiceypy.furnsh("../kernels/pck/gm_de431.tpc")

Get the location of the Earth relative to the sun

In [8]:
date_today = datetime.datetime.today()
date_today_str = date_today.strftime("%Y-%m-%dT00:00:00")
et_today_midnight = spiceypy.utc2et(date_today_str)
# Get 6-dimensional vector position of Earth
earth_state_wrt_sun, earth_sun_light_time = spiceypy.spkgeo(
    targ=399, et=et_today_midnight, ref="ECLIPJ2000", obs=10
)

# print(date_today_str)
# print(earth_state_wrt_sun)

2023-06-12T00:00:00
[-2.51047034e+07 -1.49808876e+08  8.17376334e+03  2.88957915e+01
 -5.04873320e+00 -8.79847997e-04]


Find the distance and velocity of the Earth

In [9]:
# Create a numpy array of the Earth location information
earth_state_wrt_sun = np.array(earth_state_wrt_sun)
# Compute the distance of the Earth from the sun (km)
earth_sun_distance = np.linalg.norm(earth_state_wrt_sun[:3])
print(earth_sun_distance)
# Earth orbit velocity (km/s)
earth_orb_speed_wrt_sun = np.linalg.norm(earth_state_wrt_sun[3:])
print(earth_orb_speed_wrt_sun)

151897812.94880846
29.333538339020805


In [5]:
# Extract an array with the mass times the Gravity constant of the sun (GM)
_, GM_SUN = spiceypy.bodvcd(bodyid=10, item="GM", maxn=1)
# Define a function to determine velocity of an orbiting body
v_orb_func = lambda gm, r: np.sqrt(gm / r)
# Define the theoretical velocity of a body orbiting the sun
earth_orbit_speed_speed_wrt_sun_theory = v_orb_func(gm=GM_SUN[0], r=earth_sun_distance)

print(earth_orbit_speed_speed_wrt_sun_theory)

29.55834110744839

In [6]:
# Array of Earth's position values
earth_position_wrt_sun = earth_state_wrt_sun[:3]
# Convert distance values to AU
earth_position_wrt_sun_normed = earth_position_wrt_sun / earth_sun_distance
# Create array of position at autumnal equinox, x=1, y=0,z=0
earth_position_wrt_sun_normed_autumn = np.array([1.0, 0.0, 0.0])

In [7]:
# Compute the current angular distance from the Earth's location at the autumnal equinox
ang_dist_deg = np.degrees(
    np.arccos(
        np.dot(earth_position_wrt_sun_normed, earth_position_wrt_sun_normed_autumn)
    )
)
ang_dist_deg

99.51313071886523